<a href="https://colab.research.google.com/github/huzefa-mehta/orgai/blob/main/GraphNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import os
import re
import csv
import sys, getopt
import warnings
!pip install nbconvert
warnings.filterwarnings('ignore')
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt
from spacy import displacy
!python3 -m spacy download en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()
#!python -m spacy download en_core_web_lg
#import en_core_web_lg
#nlp = spacy.load('en_core_web_lg')
from enum import Enum
G = nx.Graph()
class Relation(Enum):
  PARTOF = 1
  REPORTSTO = 2
  NOTREPORTTO = 3
  NOTPARTOF = 4
  CHANGEREPORTTO = 5
  CHANGEPARTOF = 6
  ADD = 7
  REMOVE = 8
  TITLE = 9
  PARTOFHEADS = 10

relationString = {}
relationString[Relation.PARTOF] = "partof"
relationString[Relation.REPORTSTO] = "reportsto"
debug = False
critical_list = ['DV', 'Graphics']



✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
%time

def is_csv(infile):
    try:
        with open(infile, newline='') as csvfile:
            start = csvfile.read(4096)

            # isprintable does not allow newlines, printable does not allow umlauts...
            if not all([c in string.printable or c.isprintable() for c in start]):
                return False
            dialect = csv.Sniffer().sniff(start)
            return True
    except csv.Error:
        # Could not get a csv dialect -> probably not a csv.
        return False
        
def getSentences(text):
    nlp = English()
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    document = nlp(text)
    #for sent_i, sent in enumerate(document.sents):
    #  for token in sent:
    #    print(sent_i, token.i, token.text)
    return [sent.string.strip() for sent in document.sents]

company_members = []
member_id=0

def is_critical(name):
  if name in critical_list:
    return True
  return False

def delete_member(name):
  global company_members
  for member in company_members:
      print(member)
      if(member["fn"] == str(name) or member["ln"] == str(name) or member["nn"] == str(name)):
        print(name, member)
        company_members.remove(member)
        return
  return

def find_member(name):
  global company_members
  names = name.split()
  for member in company_members:
    if(member["fn"] == str(names[0]) or member["ln"] == str(names[0]) or member["nn"] == str(names[0])):
      return member
  if len(names) > 1:
    for member in company_members:
      if(member["fn"] == str(names[1]) or member["ln"] == str(names[1]) or member["nn"] == str(names[1])):
        return member
  return None

def find_member_old(name):
  global company_members
  for member in company_members:
    if(member["fn"] == str(name) or member["ln"] == str(name) or member["nn"] == str(name)):
      n = member["fn"]+" "+member["ln"]
      n = n.strip()
      return member["id"], str(n.strip())
  return -1, name



def add_member(fn, ln="",nn="",org="", title="", reports=[], critical=False):
  global member_id
  global company_members
  member = find_member(fn)
  if(member == None):
    person = {
      "id": int(member_id),
      "fn": str(fn),
      "ln": str(ln),
      "nn": str(nn),
      "org": str(org),
      "title": str(title),
      "reports": list(reports),
      "critical": critical
    }
    if debug:
      print(person)
    company_members.append(person)
    member_id=member_id+1

def print_members():
  global company_members
  for member in company_members:
    print(member)  




node_dist_to_color = {
    1: "tab:red",
    2: "tab:orange",
}
edge_colors=[]
prons = ['he', 'she', 'him', 'his', 'her', 'they', 'them', 'their']
titles = ['CEO', 'Head', 'Director', 'Architect']
def printGraph():

    pos = nx.spring_layout(G)
    plt.figure()
    node_opts = {"node_size": 500, "node_color": "w", "edgecolors": "k", "linewidths": 2.0}
    nx.draw_networkx_nodes(G, pos, **node_opts)
    nx.draw_networkx_labels(G, pos, font_size=14)
    nx.draw_networkx_edges(G, pos, width=2.0, arrowsize=20, arrowstyle='fancy', edge_color=edge_colors)
    #nx.draw(G, pos, edge_color='black', width=1, linewidths=1,
    #        node_size=500, node_color='seagreen', alpha=0.9,
    #        labels={node: node for node in G.nodes()})
    plt.axis('off')
    plt.show()

def nodeName(name):
  member = find_member(name)
  sname = name.strip()
  if(member != None):
      if(member["ln"] != ""):
        sname=member["fn"]+" "+member["ln"]
      else:
        sname=member["fn"]
      sname=sname.strip()
      return(sname)
  return(sname)

def addNode(name):
  member=find_member(name)
  fname = name
  if(member == None):
    names = name.split()
    if len(names) > 1:
        add_member(names[0], names[1])
    else:
        add_member(names[0])
    G.add_node(name)
  else:
    fname=member["fn"]+" "+member["ln"]
    fname=fname.strip()
  return(fname)
  

def removeRelation(s,d):
  print("Removing ", s,d)
  sname=nodeName(s)
  dname=nodeName(d)
  member = find_member(sname)
  if member != None:
    reports = member["reports"]
    if reports != []:
      reports.remove(dname)
  member = find_member(sname)
  if member != None:
    reports = member["reports"]
    if reports != []:
      reports.remove(sname)

  if(G.has_edge(s,d)):
    G.remove_edge(s,d)


def addRelation(s, d, r, critical):
  print("Adding ", s,d,r,critical)
  sname = s
  dname = d
  if(r == Relation.TITLE or r == Relation.PARTOFHEADS or r == Relation.PARTOF):
    sname = addNode(s)
    member = find_member(sname)
    d=d.strip()
    member["critical"] = member["critical"] or critical
    if(r == Relation.PARTOF):
      member["org"] = d
    elif(r == Relation.TITLE):
      member["title"] = d
    else:
      member["org"] = d
      member['title'] = "Head"
    return

  if(r == Relation.REPORTSTO):
    sname = addNode(s)
    dname = addNode(d)
    member = find_member(dname)
    reports = member["reports"]
    reports.append(sname)
    member["reports"] = reports
    #G.add_edge(sname, dname)
    return



  if(r == Relation.REMOVE):
    sname = nodeName(s)
    delete_member(sname)
    if(G.has_node(sname)):
      G.remove_node(sname)

    return



  if(r == Relation.REPORTSTO):
    edge_colors.append("tab:red")
  elif(r == Relation.PARTOF):
    edge_colors.append("tab:blue")





prev_doc = None
prev_antecedent = None

def find_person(doc, t):
  for ent in doc.ents:
    if ent.label_ == 'PERSON' and re.search(t.text, ent.text): 
      return ent
  return t

def is_title(t):
  tlower = t.text
  tlower = tlower.lower()
  for item in titles:
    if tlower == item.lower():
      return True
  return False

def is_unknown(t):
  if(t.similarity(nlp("unknown")) > 0.6):
    return(True)
  if(t.similarity(nlp("rumor")) > 0.6):
    return(True)
  if(t.pos_ == "VERB" and t.similarity(nlp("remains")) > 0.6):
    print("remains",t)
    return(True)
  if(t.pos_ == "VERB" and t.similarity(nlp("continue")) > 0.6):
    print("continue",t)
    return(True)
  return(False)

def addTitleOrg(member):
  if(member == None):
    return("")
  memberName = member["fn"]+" "+member["ln"]
  s = memberName.strip()
  paren=False
  if member["title"] != "":
    paren=True
    s=s+"("+member["title"]
    if member["org"] != "":
      if(paren==False):
        s=s+"("
        paren = True
      s=s+" "+member["org"]

    if member["critical"]:
      if(paren==False):
        s=s+"("
        paren = True
      s=s+" *"
    if paren:
      s=s+")"
  s=s.strip()
  return(s)

def graph2csv(filename):
  f = open(filename, "w")
  global company_members
  for member in company_members:
    if(member["title"] == "CEO"):
      mname=member["fn"]+" "+member["ln"]
      mname=mname.strip()
      s=mname+"("+member["title"]+"),\n"
      f.write(s)
  for member in company_members:
    #mname=member["fn"]+" "+member["ln"]
    #mname=mname.strip()
    mname = addTitleOrg(member)
    for reports in member["reports"]:
      reportmem = find_member(reports)
      s = addTitleOrg(reportmem)
      s=s+","+mname+"\n"
      f.write(s)
  f.close()
    


def find_antecedent(sent_prev):
  for ent in sent_prev.ents:
    if ent.label_ == 'PERSON':
      return ent
  return None

def relationship(doc):
  if(doc.similarity(nlp("reports")) > 0.6):
    return Relation.REPORTSTO
  elif(doc.similarity(nlp("part of")) > 0.6):
    return Relation.PARTOF
  elif(doc.similarity(nlp("responsible")) > 0.6):
    return Relation.PARTOFHEADS
  elif(doc.similarity(nlp("departure")) > 0.6):
    return Relation.REMOVE
  elif(doc.similarity(nlp("join")) > 0.6):
    return Relation.ADD
  elif(doc.similarity(nlp("moved")) > 0.6):
    return Relation.CHANGEPARTOF
  elif(doc.similarity(nlp("be")) > 0.6):
    return Relation.TITLE
  elif(doc.similarity(nlp("charge")) >0.6):
    return Relation.PARTOFHEADS
  elif(doc.similarity(nlp("heads")) > 0.6):
    return Relation.PARTOFHEADS
  else:
    return None

def is_relevant(doc):
  global prev_doc
  global prev_antecedent
  global debug
  if debug:
    #displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})
    print(doc)
  subject = False
  object = False
  verb = False
  subjects = []
  objects = []
  fromobjects = []
  toobjects = []
  root_sub = None
  root_obj = None
  relation = None
  fromchange=False
  tochange=False
  fromobject = False
  toobject = False
  critical = False

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""



  for t in doc:
    ent = None
    if (is_critical(t.text)):
      critical = True
    if (is_unknown(t)):
      return False
    if relation == Relation.CHANGEPARTOF and t.pos_ == 'ADP' and t.similarity(nlp("from")) > 0.6:
      fromchange=True
    if relation == Relation.CHANGEPARTOF and (t.pos_ == 'ADP' or t.pos_ == 'PART') and t.similarity(nlp("to")) >0.6:
      tochange=True

    if t.dep_ == "compound":
      prefix = t.text
        # if the previous word was also a 'compound' then add the current word to it
      if prv_tok_dep == "compound":
        prefix = prv_tok_text + " "+ t.text
      
      # check: token is a modifier or not
    if t.dep_.endswith("mod") == True:
      modifier = t.text
      # if the previous word was also a 'compound' then add the current word to it
      if prv_tok_dep == "compound":
        modifier = prv_tok_text + " "+ t.text
    if t.pos_ == 'PROPN' or t.pos_ == 'PRON' or t.pos_ == 'NOUN' or t.pos_ == 'VERB' or t.pos_ == 'ADJ':
      ent = None


      if t.dep_ == 'nsubj' or t.dep_ == 'nsubjpass':
        if t.pos_ == 'PROPN':
          ent = find_person(doc, t)
          ent = t
          root_sub = ent
        elif t.pos_ == 'PRON':
          if (t.tag_ == 'PRP' or t.tag_ == 'PRP$') and prev_doc != None and t.text.lower() in prons:
            ent = find_antecedent(prev_doc)
            if (ent != None):
              prev_antecedent = ent
            elif(prev_antecedent != None):
              ent = prev_antecedent
        if (ent != None):
          subject = True
          #root_sub = ent

      if t.dep_ == 'conj' and t.pos_ == 'PROPN' and root_sub != None and root_sub.is_ancestor(t):
        subject = True
        ent = t


      if subject == True and ent != None:
        #subjects.append(ent.text)
        str = modifier+" "+prefix+" "+ent.text
        subjects.append(str)
        modifier=""
        prefix=""

      
      if t.dep_ == 'pobj' or t.dep_ == 'attr' or t.dep_ == 'xcomp':
        if t.pos_ == 'PROPN' or t.pos_ == 'NOUN':
          ent = find_person(doc, t)
          ent = t


          if is_title(t):
            verb = True
            relation = Relation.TITLE

          if relationship(t) != None:
            relation = relationship(t)

            verb = True
          else:
            root_obj = ent
        elif t.pos_ == 'PRON':
          if (t.tag_ == 'PRP' or t.tag_ == 'PRP$') and prev_doc != None and t.text.lower() in prons:
            ent = find_antecedent(prev_doc)
            if (ent != None):
              prev_antecedent = ent
            elif(prev_antecedent != None):
              ent = prev_antecedent
        if tochange == True and ent != None:
          toobject=True
        elif fromchange == True  and ent != None:
          fromobject=True
        elif relationship(t) == None and ent != None:
          object = True
          #root_obj = ent

      if t.dep_ == 'conj' and root_obj != None and root_obj.is_ancestor(t):
        if tochange == True:
          toobject = True
        elif fromchange == True:
          fromobject = True
        else:
          object = True
        ent = t
      
      if (t.pos_ == 'NOUN' or t.pos_ == 'VERB' or 
          (t.pos_ == 'ADJ' and t.dep_ == 'acomp')) and relationship(t):
        relation = relationship(t)
        verb = True
        if(relation == Relation.ADD or relation == Relation.REMOVE):
          object = True
    
      if toobject == True and ent != None:
        toobjects.append(ent.text)
      elif fromobject == True and ent != None:
        fromobjects.append(ent.text)
      elif object == True and ent != None:
        str = modifier+" "+prefix+" "+ent.text
        objects.append(str)
        modifier=""
        prefix=""
    else:
      modifier = ""
      prefix = ""
      prv_tok_dep = ""    # dependency tag of previous token in the sentence
      prv_tok_text = ""
  
  if subject and (object or (fromobject and toobject)) and verb:
    if(relation == Relation.CHANGEPARTOF or relation == Relation.CHANGEREPORTTO):
      for i in subjects:
        for j in toobjects:
          addRelation(i, j, relation, critical)
      for i in subjects:
        for j in fromobjects:
          removeRelation(i, j)
    elif(relation == Relation.ADD or relation == Relation.REMOVE):
      for i in subjects:
        addRelation(i, None, relation, critical)
    elif(relation == Relation.PARTOF or relation == Relation.REPORTSTO):
      for i in subjects:
        for j in objects:
          addRelation(i, j, relation, critical)
    elif(relation == Relation.TITLE):
      for i in subjects:
        for j in objects:
          addRelation(i, j, relation, critical)
    elif(relation == Relation.PARTOFHEADS):
      for i in subjects:
        for j in objects:
          addRelation(i, j, relation, critical)

    


  return subject and (object or (fromobject and toobject)) and verb

def processTranscript(transcript,csvFile=""):
  global prev_doc
  global prev_antecedent
  global debug
  prev_doc=None
  prev_antecedent=None
  sentences = getSentences(transcript)
  for doc in nlp.pipe(sentences):
    t=is_relevant(doc)
    if debug:
      print(t)
      #printGraph()
    prev_doc=doc
  if(csvFile != ""):
    graph2csv(csvFile)









def addInitialGraph():
  global debug
  debug=False
  initialTranscript = "RS, AL and AR report to BK. \
   BK reports to Raja Koduri. \
   Jim Keller, DH and RT report to Murthy. \
   Murthy reports to Bob Swan. \
   RS is responsibile for 'CoS'. \
   AR is in charge of Harwdware. \
   AL is in charge of Silicon. \
   Raja Koduri reports to Bob Swan. \
   Bob Swan is the CEO. \
   Raja Koduri is in charge of Graphics. \
   JN reports to KT. \
   Greg Bryant and Rob Crooke report to Bob Swan. \
   KT is in charge of Central Cad. \
   KT and AZ report to DH. \
   SS reports to Tom Lantzh. \
   Navin Shenoy reports to Bob Swan \
   Jim Keller is in charge of Soc Eng. \
   SM, MH and BP report to Jim Keller. \
   SM is in charge of Configurable IP. \
   Tom Lantzh reports to Murthy. \
   SB reports to AM. \
   AM reports to SM. \
   SB is the technical architect. \
   KC, RK and UK report to Navin Shenoy. \
   Navin heads Server Biz. \
   Amnon Shashua heads MobileEye. \
   Amnon Shashua reports to Bob Swan. \
   MD reports to EG. \
   EG, RR and RO report to RG. \
   US and RG report to RT. \
   JZ is in charge of Soft IP. "
  processTranscript(initialTranscript, "initial.csv")
  #if debug:
  print_members()

def processGraph():
  global debug
  debug = True

  transcript = "Lots of changes. "\
                "Murthy is gone, following Jim’s departure. "\
                "Raja now reports directly to Bob. "\
                "He remains responsible for Graphics, Architecture and Software. "\
                "BK and team continue reporting to him. "\
                "Multiple teams now reporting to Bob directly instead of via Murthy. "\
                "Tom Lantzsch’s IOT team among them."\
                "Sandra moved from Networking to become HR head."\
                "Amnon remains on Bob’s team, continues heading Mobileye."\
                "Navin and Greg continue to run respective Data Platforms and Client business groups."\
                "Randhir’s position is elevated, still running GSM but under Bob."\
                "Josh Walden has mostly replaced Murthy running significant part of engineering execution."\
                "Need to connect with AL on BK’s team, discuss new graphics architecture." \
                "SM, BP and MH all report to Josh now, following Jim’s departure, and reversal of org announcement with DH’s role. "\
                "Rumored that DH will leave Intel."\
                "AM and SB reporting structure to SM remains the same. AM remains SM’s trusted lieutenant, and relies on SB for frontend expertise"\
                "Need to stay close to RS for IP opportunities on BP’s team."\
                "New changes could be rough for SS on Tom’s IOT team. He might follow Murthy."\
                "Expect AR on SS’s staff to gain power."\
                "KT will report to Josh running infrastructure. JN is key verification lead in KT org."\
                "AZ will move under SM, centralizing the IP dev org."\
                "Spoke with US on Randhir’s staff. "\
                "He will discuss next MRM with EG. Agenda to be determined. "\
                "Need to connect NZ (Cadence) with RG prior to Randhir meeting, will align prep with EG. "\
                "Also connect AD (Cadence) with NM for roadmap prior to MRM."\
                "LT (Cadence) met with Navin to discuss datacenter investments. "\
                "EDA follow up needed with RK and KV. UK is software only."

  processTranscript(transcript, "revised.csv")
  if debug:
    print_members()

def run():
  os.environ["SPACY_WARNING_IGNORE"] = "W008"
  G = nx.Graph()
  company_members = []
  addInitialGraph()
  processGraph()

run()



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 10.7 µs
Adding    RS   BK Relation.REPORTSTO False
Adding    AL   BK Relation.REPORTSTO False
Adding    AR   BK Relation.REPORTSTO False
Adding    BK  Raja Koduri Relation.REPORTSTO False
Adding   Jim Keller   Murthy Relation.REPORTSTO False
Adding    DH   Murthy Relation.REPORTSTO False
Adding    RT   Murthy Relation.REPORTSTO False
Adding    Murthy  Bob Swan Relation.REPORTSTO False
Adding    RS   CoS Relation.PARTOFHEADS False
Adding    AR   Harwdware Relation.PARTOFHEADS False
Adding    AL   Silicon Relation.PARTOFHEADS False
Adding   Raja Koduri  Bob Swan Relation.REPORTSTO False
Adding   Bob Swan   CEO Relation.TITLE False
Adding   Raja Koduri   Graphics Relation.PARTOFHEADS True
Adding    JN   KT Relation.REPORTSTO False
Adding   Greg Bryant  Bob Swan Relation.REPORTSTO False
Adding   Rob Crooke  Bob Swan Relation.REPORTSTO False
Adding    KT  Central Cad Relation.PARTOFHEADS False
Adding    KT   DH Relation.REPORTSTO False

In [ ]:


def processFile(fname,csvFile):
  fname = fname.strip()
  if not is_csv(fname):
    transcript=""
    with open(fname, 'r') as f:
      transcript = f.read().replace('\n', '')

    f.close()
    if(transcript):
      processTranscript(transcript,csvFile)




progname = "orgai.py"
def main(argv):
   global critical
   critical_list = []
   initial_transcript = ''
   revised_transcript = ''
   progname = "orgai.py"
   argv = argv[1:]
   try:
      opts, args = getopt.getopt(argv,"hi:r:c:",["ifile=","rfile=","critical="])
   except getopt.GetoptError:
      print(progname, ' -i <initialtranscript> -r <revisedtranscript> -c <critical>')
      return
   for opt, arg in opts:
      if opt == '-h':
         print(progname, '-i <initialtranscript> -r <revisedtranscript> -c <critical>')
         return
      elif opt in ("-i", "--ifile"):
         initial_transcript = arg
      elif opt in ("-r", "--rfile"):
         revised_transcript = arg
      elif opt in ("-c", "--critical"):
         critical_list = arg


   print(critical_list)
   if initial_transcript != '' and revised_transcript != '':
        processFile(initial_transcript, "initial.csv")
        processFile(revised_transcript, "revised.csv")
   else:
        print(progname, '-i <initialtranscript> -r <revisedtranscript>')


   
#orgai.py -i initial_transcript.txt -r revised_transcript.txt -c ["graphics", "soc", "budget", "system"]
if __name__ == "__main__":
  #main(["orgai.py","-i /content/initial_transcript", "-r /content/revised_transcript", "-c ['Graphics', 'SOC']"])
  main(["orgai.py","-i", "/content/initial_transcript", "-r", "/content/revised_transcript", "-c", "[SOC]"])
  #main(sys.argv)
  

[SOC]


FileNotFoundError: ignored

In [ ]:

#!jupyter nbconvert --to GraphNLP.py GraphNLP.ipynb
#!pip install pyinstaller
#!pyinstaller GraphNLP.py

bin	 datalab  home	 lib64	opt   run   swift	       tmp    var
boot	 dev	  lib	 media	proc  sbin  sys		       tools
content  etc	  lib32  mnt	root  srv   tensorflow-1.15.2  usr
[NbConvertApp] WARNING | pattern u'GraphNLP.ipynb' matched no files
[NbConvertApp] ERROR | Error importing GraphNLP.py
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/nbconvert/exporters/base.py", line 107, in get_exporter
    return import_item(name)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/utils/importstring.py", line 34, in import_item
    module = __import__(package, fromlist=[obj])
ImportError: No module named GraphNLP
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/tra

In [ ]:
!pip install gspread
!pip install oauth2client
from google.colab import drive
drive.mount('/content/drive')
!pwd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive", "/content"]
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/OrgAI-5ce11d05f14a.json', scope)
print(credentials)
client = gspread.authorize(credentials)

spreadsheet = client.open('CSV-to-Google-Sheet')

with open('data.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)


KeyboardInterrupt: ignored